<a href="https://colab.research.google.com/github/Ahtesham519/Genrative_Deep_learning_v2_2023/blob/main/Autoencoder_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow.keras import(
    layers,
    models,
    callbacks,
    utils,
    metrics,
    losses,
    optimizers,
)

from scipy.stats import norm
import pandas as pd

from vae_utils import get_vector_from_label , add_vector_to_images, morph_faces

#0. Parameters

In [ ]:
IMAGE_SIZE = 32
CHANNELS = 3
BATCH_SIZE = 128
NUM_FEATURES = 128
Z_DIM = 200
LEARNING_RATE = 0.0005
EPOCHS = 10
BETA = 2000
LOAD_MODEL = False


#1. Prepare the data

In [ ]:
#Load the data
train_data = utils.image_dataset_from_directory(
    "/app/data/celeba-dataset/img_align_celeba/img_align_celeba",
    labels = None,
    color_mode = "rgb",
    image_size = (IMAGE_SIZE , IMAGE_SIZE),
    batch_size = BATCH_SIZE ,
    shuffle = True,
    seed = 42,
    interpolation = "bilinear",
)

In [ ]:
#preprocess the data
def preprocess(img):
  img = tf.cast(img, "float32") / 255.0
  return img

train = train_data.map(lambda x: preprocess(x))

In [ ]:
train_sample = sample_batch(train)

In [ ]:
#Show some faces from the training set


#2. Bulid hte varaitional autoencoder

In [ ]:
class Sampling(layers.Layer):
  def call(self, inputs):
    z_mean , z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilion = K.random_normal(shape = (batch , dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon


In [ ]:
#encoder
encoder_input = layers.Input(
    shape = (IMAGE_SIZE , IMAGE_SIZE , CHANNELS), name = "encoder_input"
)
x = layers.Conv2D(NUM_FEATURES , kernel_size = 3 , strides = 2, padding = "same")(
    encoder_input
)
x = layers.BatchNormalization()(x)
x = layers.LEakyReLU()(x)
x = layers.Conv2D(NUM_FEATURES, kernel_size = 3 , strides = 2, padding = "same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(NUM_FEATURES , kernel_size = 3, strides = 2, padding = "same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(NUM_FEATURES , kernel_size = 3, strides = 2, padding ="same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
shape_before_flattening = K.int_shape(x)[1:]

x = layers..Flattening()(x)
z_mean = layers.Dense(Z_DIM , name = "z_mean")(x)
z_log_var = layers.Dense(Z_DIM , name = "z_log_var")(x)
z = Sampling()([z_mean , z_log_var])

encoder = models.Model(encoder_input , [z_mean , z_log_var, z], name = "encoder")
encoder.summary()

In [ ]:
#decoder
decoder_input = layers.Input(shape =(Z_DIM,), name = "decoder_input")
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Reshape(shape_before_flattening)(x)
x = layers.Conv2DTranspose(
    NUM_FEATURES , kernel_size = 3, strides = 2, padding = "same"
)

x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(
    NUM_FEATURES , kernel_size = 3, strides = 2, padding = "same"
)(x)

x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(
    NUM_FEATURES , kernel_size = 3, strides = 2, padding = "same"
)(x)

x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(
    NUM_FEATURES , kernel_size = 3, strides = 2, padding = "same"
)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
decoder_ouput = layers.Conv2DTranspose(
    CHANNELS , kernel_size = 3 , strides = 1, activation = "sigmoid" , padding = "same"
)(x)
decoder = models.Model(decoder_input, decoder_output)
decoder.summary()

In [ ]:
class VAE(models.Model):
  def __init__(self, encoder, decoder, **kwargs):
    super(VAE, self).__init__(**kwargs)
    self.encoder = encoder
    self.decoder = decoder
    self.total_loss_tracker = metrics.Mean(name = "total_loss")
    self.reconstruction_loss_tracker = metrics.Mean(
        name = "reconstruction_loss"
    )
    self.kl_loss_tracker = metrics.Mean(name= "kl_loss")

    @property
    def metrics(self):
      return [
          self.total_loss_tracker,
          self.reconstruction_loss_tracker,
          self.kl_loss_tracker,
      ]

    def call(self, inputs):
      """ Call the model on a particular input."""
      z_mean , z_log_var , z = encoder(inputs)
      reconstruction = decoder(z)
      return z_mean , z_log_var , reconstruction

    def train_step(self, data):
      """Step run during training"""
      with tf.GradientTape() as tape:
        z_mean , z_log_var , reconstruction = self(data, training = True)
        reconstruction_loss = tf.reduce_mean(
            BETA * losses.mean_squared_error(data, reconstruction)
        )
        kl_loss = tf.reduce_mean(
            tf.reduce_sum(
                -0.5,
                * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)),
                axis = 1,
            )
        )
        total_loss = reconstruction_loss + kl_loss

      grads = tape.gradient(total_loss , self.trainable_wights)
      self.optimizer.apply_gradients(zip(grads , self.trainable_weights))

      self.total_loss_tracker.update_state(total_loss)
      self.reconstruction_loss_tracker.update_state(reconstruction_loss)
      self.kl_loss_tracker.update_state(kl_loss)


      return {
          "loss" : self.total_loss_tracker.result(),
          "reconstruction_loss": self.reconstruciton_loss_tracker.result(),
          "kl_loss": self.kl_loss_tracker,.result(),
      }

    def test_step(self, data):
      """Step run during validation"""

      if isinstance(data, tuple):
        data = data[0]

      z_mean , z_log_var, reconstruction = self(data)
      reconstruction_loss = tf.reduce_mean(
          BETA * losses.mean_squared_error(data, reconstruction)
      )
      kl_loss = tf.reduce_mean(
          tf.reduce_sum(
              -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)),
              axis = 1,
          )
      )
      total_loss = reconstruction_loss + kl_loss

      return{
          "loss": total_loss,
          "reconstruction_loss" : reconstruction_loss,
          "kl_loss": kl_loss,
      }


In [ ]:
#Create a varational autoencoder
vae = VAE(encoder, decoder)

#3. Train the variational autoencoder

In [ ]:
#compile the varational encoder
optimizer = optimizers.Adam(learning_rate  = LEARNING_RATE)
vae.compile(optimizer = optimizer)

In [ ]:
#create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath = "./checkpoint",
    save_weight_only = False,
    save_freq = "epoch",
    monitor = "loss",
    mode = "min",
    save_best_only = True,
    verbose = 0,
)


tensorboard_callback = callbacks.TensorBoard(log_dir = "./logs")

class ImageGenerator(callbacks.Callback):
  def __init__(self, num_img, latent_dim):
    self.num_img = num_img
    self.latent_dim = latent_dim

  def on_epoch_end(self, epoch , logs = None):
    random_latent_vectors = tf.random.normal(
        shape = (self.num_img , self.latent_dim)
    )
    generated_images = self.model.decoder(random_latent_vectors)
    generated_images *= 255
    generated_images.numpy()
    for i in range(self.num_img):
      img = utils.array_to_img(generated_images[i])
      img.save("./output/generated_img_%03d_%d.png" % (epoch , i))

In [ ]:
#Load old weights if required
if LOAD_MODEL:
  vae.load_weights("./models/vae")
  tmp = vae.predict(train.take(1))

In [ ]:
vae.fit(
    train,
    epochs = EPOCHS,
    callbacks = [
        model_checkpoint_callback,
        tensorboard_callback,
        ImageGenerator(num_img = 10, latent_dim = Z_DIM),
    ],
)

In [ ]:
#save the final models
vae.save("./models/vae")
encoder.save("./models/encoder")
decoder.save("./models/decoder")

#3. Reconstruciton using the variational autoencoder

In [ ]:
#Select a subset of the test set
batched_to_predict = 1
example_images = np.array(
    list(train.take(batches_to_predict).get_single_element())
)

In [ ]:
#create autoencoder predictions and display
z_mean , z_log_var , reconstructions = vae.predict(example_images)
print("Example real faces")
display(example_images)
print("Reconstructions")
display(reconstructions)

#4. Latent space distribution

In [ ]:
_, _, z  =vae.encoder.predict(example_images)

x = np.linspace(-3, 3, 100)

fig = plt.figure(figsize = ( 20 , 5))
fig.subplots_adjust(hspace = 0.6 , wspace = 0.4)

for i in range(50):
  ax = fig.add_subplots(5, 10 , i + 1)
  ax.hist(z[:, i], density = True, bins = 20)
  ax.axis("off")
  ax.text(
      0.5, -0.35, str(i) , fontsize = 10 , ha = "center" , transform = ax.transAxes
  )
  ax.plot(x, norm.pdf(x))


plt.show()

#5. Generate new faces

In [ ]:
#sampling some points in the latent space ,from the standard normal distribution
grid_width , grid_height = (10 , 3)
z_sample = np.random.normal(size = (grid_width * grid_height , Z_DIM))


In [ ]:
#decode the smaples points
reconstrucitons = decoder.predict(z_sample)

In [ ]:
#Draw a plot of decoder images
fig = plt.figure(figsize = (18, 5))
fig.subplots_adjust(hspace = 0.4 , wspace = 0.4)

#output the grid of faces
for i in range(grid_width * grid_height):
  ax = fig.add_subplots(grid_height , grid_weight, i + 1)
  ax.axis("off")
  ax.imshow(reconstructions[i , : , :])

#6. Manipulate the images

In [ ]:
#load the label dataset

attributes = pd.read_csv("./app/data/celeba-dataset/list_attr_celeba.csv")
print(attributes.columns)
attributes.head()

In [ ]:
#load the face data with label attached
LABEL = "BLOND_Hair"
labelled_test = utils.image_dataset_from_directory(
    "./app/data/celeba-dataset/img_align_celeba",
    labels = attributes[LABEL].tolist(),
    color_mode = "rgb",
    image_size = (IMAGE_SIZE , IMAGE_SIZE),
    batch_size = BATCH_SIZE ,
    shuffle = True,
    seed = 42,
    validation_split = 0.2,
    subset = "validation",
    interpolation = "bilinear",
)

labelled = labelled_test.map(lambda x, y : (preprocess(x), y))

In [ ]:
#Find the attributes vector
attrubute_vec = get_vector_from_label(labelled, vae, Z_DIM, LABEL)


In [ ]:
#Add vector to images
add_vector_to_images(labelled , vae , attribute_vec)

In [ ]:
morph_faces(labelled , vae)